## Q3. (40%) You are hired by an officer from the WFP (World Food Programme, the food-assistance branch of the United Nations) to assess the drought condition for Guatemala (GT) in Central America. The result of your assessment is very important because it would impact the amount of funding that WFP will allocate to help GT in the following few years. 

## After some literature review, you decide to use Standard Precipitation Index (SPI) for the assessment. The SPI is a widely-used index to quantify the precipitation deficit for multiple timescales (WMO, 2012). These timescales reflect the impact of drought on the availability of the different water resources. For example, Soil moisture conditions respond to precipitation anomalies on a relatively short scale (approximately 1 month). Groundwater, streamflow and reservoir storage reflect the longer-term precipitation anomalies (usually 3+ months). The underlying idea of SPIs is to quantify observed rainfall as a standardized departure from a selected probability distribution function that models the raw precipitation data. 

## The key to obtain reliable SPIs is to look for a suitable probability distribution for the underlying data. So the task here is to implement a tool that can automatically compare several candidate distributions and identify the best one.


### 3.0. Import essential modules

In [1]:
import pandas as pd
import scipy.stats as stats

### 3.1. (5%) Import daily rainfall estimates for all locations of interest for the period of 1979 - 2020 from the attached CSV file _Rain_1d_GT_19790101-20201231.csv_ in the _Data_ folder. 

### Then, for each location, compute rainfall estimates for each _calendar_ month, and export the monthly rainfall to a CSV file named _Rain_1M_GT_19790101-20201231.csv_ where the first row is the header, and, for the following rows, each row includes rainfall estimates for a given month for all locations of interest. The format example is as follows (to 6 decimal places:

date,101,102,...</br>
1979/01,...,...</br>
1979/02,...,...</br>
...</br>
2020/12,...,...</br>

In [2]:
path='Data/Rain_1d_GT_19790101-20201231.csv'
df = pd.read_csv(path)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
month_rainfall = df.resample('M').sum()
month_rainfall.index = month_rainfall.index.to_period("M")
month_rainfall = month_rainfall.round(6)
month_rainfall_output = month_rainfall.copy()
month_rainfall_output.index = month_rainfall.index.strftime('%Y/%m')
month_rainfall_output.to_csv('Rain_1M_GT_19790101-20201231.csv')
print(month_rainfall_output)

                101         102         103         104         105  \
date                                                                  
1979/01   15.494035   15.493550   15.120030   10.435103   13.189232   
1979/02   11.766926   11.067176    9.117432    5.738555    6.316651   
1979/03   33.534119   34.157334   35.751452   26.855232   29.478315   
1979/04  132.877773  132.778320  128.822646   85.630129  104.837186   
1979/05  194.495276  196.372944  195.560265  125.236401  158.046756   
...             ...         ...         ...         ...         ...   
2020/08  275.217043  282.595973  293.782628  196.650245  242.579940   
2020/09  310.613133  314.637188  320.384219  216.343887  272.089154   
2020/10  308.596825  314.534447  319.364867  204.664207  247.284762   
2020/11  189.191323  190.041139  185.188005  158.910350  156.776663   
2020/12   12.451701   12.821830   13.341694    6.870188    8.803994   

                106         107         108         109         110  ...  \


### 3.2. (35%) Use Gamma, Weibull and Normal as candidate distribuitions, and identify which distribution is the best one to fit the monthly rainfall for each calendar month and at each location by comparing the test statistics obtained from the KS test. Please output the counts for each candidate distribution that leads to the best fit for each calendar month in a CSV file, named _Rain_1M_GT_19790101-20201231_bPD-counts.csv_. The format example is as follows:

Month,gamma,exponweib,norm</br>
1,55,...,...</br>
2,...,...,...</br>
...</br>
12,...,...,...</br>

### Note: you shall use scipy.stats. exponweib for Weibull distribution, and the location parameter shall be fixed to ‘0’

In [3]:
num_station = len(month_rainfall.iloc[0,:])
matrix = [[0] * 340 for i in range(12)]

for m in range(1,13):
    for i in range(num_station):
        samples = month_rainfall.iloc[:,i][month_rainfall.index.month == m]
        gamma_param = stats.gamma.fit(samples, floc=0)
        exponweib_param = stats.exponweib.fit(samples, floc=0)
        norm_param = stats.norm.fit(samples)

        gamma_Dn_stats, gamma_p_value_stats = stats.kstest(samples, stats.gamma.cdf, gamma_param, mode='asymp')
        exponweib_Dn_stats, exponweib_p_value_stats = stats.kstest(samples, stats.exponweib.cdf, exponweib_param, mode='asymp')
        norm_Dn_stats, norm_p_value_stats = stats.kstest(samples, stats.norm.cdf, norm_param, mode='asymp')

        if (gamma_p_value_stats > exponweib_p_value_stats) and (gamma_p_value_stats > norm_p_value_stats):
            matrix[m-1][i] = 'gamma'
        elif (exponweib_p_value_stats > gamma_p_value_stats) and (exponweib_p_value_stats > norm_p_value_stats):
            matrix[m-1][i] = 'exponweib'
        elif (norm_p_value_stats > gamma_p_value_stats) and (norm_p_value_stats > exponweib_p_value_stats):
            matrix[m-1][i] = 'norm'

month_list = list(range(1, 13))
gamma_list = []
exponweib_list = []
norm_list = []

for i in range(0,12):
    gamma_list.append(sum(1 for row in matrix[i] if row == 'gamma'))
    exponweib_list.append(sum(1 for row in matrix[i] if row == 'exponweib'))
    norm_list.append(sum(1 for row in matrix[i] if row == 'norm'))

data = {
    'Month': month_list,
    'gamma': gamma_list,
    'exponweib': exponweib_list,
    'norm': norm_list
}
count_output = pd.DataFrame(data)
count_output.to_csv('Rain_1M_GT_19790101-20201231_bPD-counts.csv', index=False)
print(count_output)

    Month  gamma  exponweib  norm
0       1     55        282     3
1       2     58        266    16
2       3     98        225    17
3       4     74        264     2
4       5     79        215    46
5       6     85        124   131
6       7     61        236    43
7       8     82        240    18
8       9     49        273    18
9      10    115        196    29
10     11      9        331     0
11     12     70        240    30
